In [ ]:
import ast
import pickle
import os
import pandas as pd

#### Luigi migration. Task yet to be created

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 


pars = load_parameters()

so_dump_processed_file = pars.get('input')
output_file = pars.get('output')

input_col = pars.get('input_col')

api_doc_file = pars.get('api_doc_file')

debug = pars.get('debug')

### Utility to extract method name and line number

In [ ]:
from collections import deque

class FunctionCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
        self._pos = -1 
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @property
    def lineno(self):
        return self._pos
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._pos = node.lineno # line number
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._pos = node.lineno # line number
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FunctionCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append((callvisitor.name, callvisitor.lineno))
    return func_calls

#### Load APIDoc and Dataset

In [ ]:
# api_doc_file = '../../data-import/build_api_doc_base/api_doc.csv'
# so_dump_processed_file = '../../../data/stack-overflow/pandas-preprocessedcode-dataset-part3'
# input_col = 'PreprocessedCode3'
# output_file = '../../../data/stack-overflow/pandas-solutioncode-h1'

api_df = pd.read_csv(api_doc_file, encoding='ISO-8859-1', error_bad_lines=False)

processed_stackoverflow_df = pd.read_pickle(so_dump_processed_file)

def buildAPIDictionary(api_df):
    api_dict = dict()
    try:
        for index, row in api_df.iterrows():
            methodContext = row['Description']
            tokens = row['FullyQualifiedName'].split('.')
        
            for token in tokens:
                methodContext = str(methodContext)+' '+token
            api_dict[row['MethodName']] = methodContext
    except Exception as e:
        print('Error in method buildAPIDictionary',e)
    return api_dict

    
api_dict = buildAPIDictionary(api_df)

In [ ]:
def lookUpAPIDocForContext(method_name):
    try:
        if method_name in api_dict.keys():
            return True
        else:
            return False
    except Exception as e:
        print('Error in method lookUpAPIDocForContext', e)

### H1 Method

Look for API calls in Code Fragment:
After analyzing the stackoverflow answers, it has been
identified that, usually solutions are provided in terms of API
method calls.

In [ ]:
def H1(df):
    
    # Do not process questions
    if df.PostTypeId == 1:
        df['Solution'] = 'NA'
    
    else:
        # Parse code and inspect the function
        code_snippet = df[input_col]
        tree = ast.parse(code_snippet)        
        func_calls = get_func_calls(tree)

        solution_lines = set()
        for function, lineno in func_calls:   
            # print(function)
            tokens = function.split('.')
            method_name = tokens[len(tokens)-1]
            if lookUpAPIDocForContext(method_name):
                # Use a set to not add the same line twice
                solution_lines.add(lineno - 1)
                
        solution = []
        snippet_per_line = code_snippet.split(os.linesep)
        solution_lines = sorted(solution_lines)
        for i in solution_lines:
            solution.append(snippet_per_line[i])
                
                
        #print(solution)
        df['Solution'] = os.linesep.join(solution)
    return df


solution_df = processed_stackoverflow_df.apply(H1, axis=1)

In [ ]:
debug = True

solution_df.to_pickle(output_file)

if debug:
    solution_df.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)
